### Importing Packages

In [1]:
import itertools
import numpy as np
from Bio import Align
from Bio import SeqIO
from Bio import pairwise2
from Bio.SubsMat.MatrixInfo import blosum62
import csv
import pandas as pd
from Bio.pairwise2 import format_alignment
import math


### Reading fast files of protein sequence

In [2]:

query_seq = list(SeqIO.parse("../Data/NpInter2 Interacting Protiens/NpInterV2_Interacting_Protiens.fasta", "fasta"))
target_seq = list(SeqIO.parse("../Data/NpInter2 Interacting Protiens/NpInterV2_Interacting_Protiens.fasta", "fasta"))
#query_seq = list(SeqIO.parse("../Data/NpInter2 Interacting Protiens/sample1.fasta", "fasta"))
#target_seq = list(SeqIO.parse("../Data/NpInter2 Interacting Protiens/sample1.fasta", "fasta"))


In [3]:
print(len(target_seq))
print(len(target_seq))

145
145


### custom format_alignment to get the match, mismatch, gaps and score

In [4]:
def fn_format_alignment(align1, align2, score, begin, end): 
    #s = [] 
    #s.append("%s\n" % align1) 
    #s.append(" " * begin) 
    #matrix=blosum62
    gap=0
    mismatch=0
    match=0
    seq_len=0
    mismatch_positive=0
    #print(align1[begin:end])
    #print(align2[begin:end])
    for a, b in zip(align1[begin:end], align2[begin:end]): 
        if a == b: 
            #s.append("|")  # match 
            match+=1
        elif a == "-" or b == "-":
            
            #s.append(" ")  # gap
            gap+=1
        elif a != b:
            if (a,b) in blosum62:
                if blosum62[(a,b)] >0:
                    mismatch_positive+=1
            else:
                if blosum62[(b,a)] >0:
                    mismatch_positive+=1
            
                                                     
            #s.append(".")  # mismatch 
            mismatch+=1
        seq_len = min(len(align1[begin:end]), len(align2[begin:end]))
    #s.append("\n") 
    #s.append("%s\n" % align2) 
    #s.append("  Score=%g\n" % score)
    #s.append("  Gap=%g\n" % gap)
    #s.append("  Mismatch=%g\n" % mismatch)
    return score,match,mismatch,gap,seq_len,mismatch_positive

In [7]:
# round decimal number

import decimal

def round_down(value, decimals):
    with decimal.localcontext() as ctx:
        d = decimal.Decimal(value)
        ctx.rounding = decimal.ROUND_DOWN
        return round(d, decimals)

### Creating Empty CSV file to store the similarity results

#### Calculating similarity score and saving in csv

In [8]:

def calculate_similarity(query_seq,target_seq):
    print("started task...")
    with open("../Data/NpInter2 Interacting Protiens/Protein-Protein Sequence Similarity/Protien-Protien_Similarity.csv", "a", newline='') as csvFile:
        # csv file header
        fieldnames = ['Query_Seq_ID','Target_Seq_ID', 'Align_Score','Normalize_Score', 'Identity', 'Similarity']
        writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
        writer.writeheader()
        # loop on query seq
        count=0
        for seq_q in query_seq:
            try:
                # self alignment with query seq
                seq_q_align = pairwise2.align.localds(seq_q.seq, seq_q.seq, blosum62, -10, -0.5, one_alignment_only=True)
                # loop on target seq
                for seq_t in target_seq:
                    if seq_q.seq != seq_t.seq:
                        
                        try:
                            # self alignment with traget seq
                            seq_t_align = pairwise2.align.localds(seq_t.seq, seq_t.seq, blosum62, -10, -0.5, one_alignment_only=True)
                            # getting the score of query seq self align
                            seq_q_align_score=fn_format_alignment(*seq_q_align[0])[0]
                            # getting the score of target seq self align
                            seq_t_align_score=fn_format_alignment(*seq_t_align[0])[0]
                            

                            # the alignment b/w qry and trg seq
                            align = pairwise2.align.localds(seq_q.seq, seq_t.seq, blosum62, -10, -0.5, one_alignment_only=True)
                            # getting the score of query and target seq
                            score =fn_format_alignment(*align[0])[0]
                            # normalizing the score
                            Normalize_Score = round_down(score/(math.sqrt(seq_q_align_score)*math.sqrt(seq_t_align_score)),2)
                            # get the no of matches in query and target seq
                            matches = fn_format_alignment(*align[0])[1]
                            # getting the no of mismatches in qry and trg seq
                            mismatches = fn_format_alignment(*align[0])[2]
                            mismatch_positive=fn_format_alignment(*align[0])[5]
                            # calcualting the percent identity %.
                            seq_len = fn_format_alignment(*align[0])[4]
                            Identity = str(round_down(matches/seq_len * 100,2)) +'%' 

                            #Identity =str(round_down(matches/(matches + mismatches) * 100,2)) +'%' 
                            # calculating the similarity in %
                            #Similarity=str(round_down((matches + mismatches)/seq_len * 100,2)) + '%'
                            Similarity=str(round_down((matches + mismatch_positive)/seq_len * 100,2)) + '%'
                            writer.writerow({'Query_Seq_ID': seq_q.id.split('|')[1],                                      
                                             'Target_Seq_ID': seq_t.id.split('|')[1],
                                             'Align_Score': score, 'Normalize_Score':Normalize_Score,
                                             'Identity':Identity, 'Similarity':Similarity })
                            count +=1
                            print(count)
                            
                            print(seq_q.id.split('|')[1])
                            print('.............................')
                            
                            #print(seq_t.id.split('|')[1])
                            #print('length: ' + str(min(len(seq_q.seq), len(seq_t.seq))))
                            #print('length2: ' + str(fn_format_alignment(*align[0])[4]))
                            #print('Score: ' + str(fn_format_alignment(*align[0])[0]))
                            #print('matches: ' + str(fn_format_alignment(*align[0])[1]))
                            #print('mismatches: ' + str(fn_format_alignment(*align[0])[2]))
                            #print('mismatches+ : '+ str(mismatch_positive))
                            #print('gaps: ' + str(fn_format_alignment(*align[0])[3]))

                            #print('Identity: ' + str(Identity))
                           
                            #print('Normalize_Score: ' + str(Normalize_Score))
                            #print('Similarity: ' + str(Similarity))
                            


                            #print('.............................')
                            
                            #print(align[0])
                            #print('---------------------------------------')
                            #print(align_score)
                            #print(align_score[1])
                            #print(align_score[2])
                            #print(align_score[3])
                            #print(type(align_score))  
                            
                        except Exception as e:
                            print(e)
                            continue # doing nothing on exception
            #count+=1
            #print(count)
            #print(seq_q.id.split('|')[1] + '  Completed')
            except Exception as e:
                    print(e)
                    continue # doing nothing on exception

    print("Task Completed.....")






In [9]:
print("started....")
calculate_similarity(query_seq,target_seq)
print("completed.....")


started....
started task...
1
Q9UPQ9
.............................
2
Q9UPQ9
.............................
3
Q9UPQ9
.............................
4
Q9UPQ9
.............................
5
Q9UPQ9
.............................
6
Q9UPQ9
.............................
7
Q9UPQ9
.............................
8
Q9UPQ9
.............................
9
Q9UPQ9
.............................
10
Q9UPQ9
.............................
11
Q9UPQ9
.............................
12
Q9UPQ9
.............................
13
Q9UPQ9
.............................
14
Q9UPQ9
.............................
15
Q9UPQ9
.............................
16
Q9UPQ9
.............................
17
Q9UPQ9
.............................
18
Q9UPQ9
.............................
19
Q9UPQ9
.............................
20
Q9UPQ9
.............................
21
Q9UPQ9
.............................
22
Q9UPQ9
.............................
23
Q9UPQ9
.............................
24
Q9UPQ9
.............................
25
Q9UPQ9
...........

203
Q13148
.............................
204
Q13148
.............................
205
Q13148
.............................
206
Q13148
.............................
207
Q13148
.............................
208
Q13148
.............................
209
Q13148
.............................
210
Q13148
.............................
211
Q13148
.............................
212
Q13148
.............................
213
Q13148
.............................
214
Q13148
.............................
215
Q13148
.............................
216
Q13148
.............................
217
Q13148
.............................
218
Q13148
.............................
219
Q13148
.............................
220
Q13148
.............................
221
Q13148
.............................
222
Q13148
.............................
223
Q13148
.............................
224
Q13148
.............................
225
Q13148
.............................
226
Q13148
.............................
227
Q13148
.....

406
P08579
.............................
407
P08579
.............................
408
P08579
.............................
409
P08579
.............................
410
P08579
.............................
411
P08579
.............................
412
P08579
.............................
413
P08579
.............................
414
P08579
.............................
415
P08579
.............................
416
P08579
.............................
417
P08579
.............................
418
P08579
.............................
419
P08579
.............................
420
P08579
.............................
421
P08579
.............................
422
P08579
.............................
423
P08579
.............................
424
P08579
.............................
425
P08579
.............................
426
P08579
.............................
427
P08579
.............................
428
P08579
.............................
429
P08579
.............................
430
P08579
.....

606
Q9UHB9
.............................
607
Q9UHB9
.............................
608
Q9UHB9
.............................
609
Q9UHB9
.............................
610
Q9UHB9
.............................
611
Q9UHB9
.............................
612
Q9UHB9
.............................
613
Q9UHB9
.............................
614
Q9UHB9
.............................
615
Q9UHB9
.............................
616
Q9UHB9
.............................
617
Q9UHB9
.............................
618
Q9UHB9
.............................
619
Q9UHB9
.............................
620
Q9UHB9
.............................
621
Q9UHB9
.............................
622
Q9UHB9
.............................
623
Q9UHB9
.............................
624
Q9UHB9
.............................
625
Q9UHB9
.............................
626
Q9UHB9
.............................
627
Q9UHB9
.............................
628
Q9UHB9
.............................
629
Q9UHB9
.............................
630
Q9UHB9
.....

806
O75940
.............................
807
O75940
.............................
808
O75940
.............................
809
O75940
.............................
810
O75940
.............................
811
O75940
.............................
812
O75940
.............................
813
O75940
.............................
814
O75940
.............................
815
O75940
.............................
816
O75940
.............................
817
O75940
.............................
818
O75940
.............................
819
O75940
.............................
820
O75940
.............................
821
O75940
.............................
822
O75940
.............................
823
O75940
.............................
824
O75940
.............................
825
O75940
.............................
826
O75940
.............................
827
O75940
.............................
828
O75940
.............................
829
O75940
.............................
830
O75940
.....

...

19964
A0PJ92
.............................
19965
A0PJ92
.............................
19966
A0PJ92
.............................
19967
A0PJ92
.............................
19968
A0PJ92
.............................
19969
A0PJ92
.............................
19970
A0PJ92
.............................
19971
A0PJ92
.............................
19972
A0PJ92
.............................
19973
A0PJ92
.............................
19974
A0PJ92
.............................
19975
A0PJ92
.............................
19976
A0PJ92
.............................
19977
A0PJ92
.............................
19978
A0PJ92
.............................
19979
A0PJ92
.............................
19980
A0PJ92
.............................
19981
A0PJ92
.............................
19982
A0PJ92
.............................
19983
A0PJ92
.............................
19984
A0PJ92
.............................
19985
A0PJ92
.............................
19986
A0PJ92
.............................
19987
A0PJ9

20155
Q49AS9
.............................
20156
Q49AS9
.............................
20157
Q49AS9
.............................
20158
Q49AS9
.............................
20159
Q49AS9
.............................
20160
Q49AS9
.............................
20161
Q6PJX3
.............................
20162
Q6PJX3
.............................
20163
Q6PJX3
.............................
20164
Q6PJX3
.............................
20165
Q6PJX3
.............................
20166
Q6PJX3
.............................
20167
Q6PJX3
.............................
20168
Q6PJX3
.............................
20169
Q6PJX3
.............................
20170
Q6PJX3
.............................
20171
Q6PJX3
.............................
20172
Q6PJX3
.............................
20173
Q6PJX3
.............................
20174
Q6PJX3
.............................
20175
Q6PJX3
.............................
20176
Q6PJX3
.............................
20177
Q6PJX3
.............................
20178
Q6PJX

20346
B7ZL34
.............................
20347
B7ZL34
.............................
20348
B7ZL34
.............................
20349
B7ZL34
.............................
20350
B7ZL34
.............................
20351
B7ZL34
.............................
20352
B7ZL34
.............................
20353
B7ZL34
.............................
20354
B7ZL34
.............................
20355
B7ZL34
.............................
20356
B7ZL34
.............................
20357
B7ZL34
.............................
20358
B7ZL34
.............................
20359
B7ZL34
.............................
20360
B7ZL34
.............................
20361
B7ZL34
.............................
20362
B7ZL34
.............................
20363
B7ZL34
.............................
20364
B7ZL34
.............................
20365
B7ZL34
.............................
20366
B7ZL34
.............................
20367
B7ZL34
.............................
20368
B7ZL34
.............................
20369
B7ZL3

20537
Q3MI39
.............................
20538
Q3MI39
.............................
20539
Q3MI39
.............................
20540
Q3MI39
.............................
20541
Q3MI39
.............................
20542
Q3MI39
.............................
20543
Q3MI39
.............................
20544
Q3MI39
.............................
20545
Q3MI39
.............................
20546
Q3MI39
.............................
20547
Q3MI39
.............................
20548
Q3MI39
.............................
20549
Q3MI39
.............................
20550
Q3MI39
.............................
20551
Q3MI39
.............................
20552
Q3MI39
.............................
20553
Q3MI39
.............................
20554
Q3MI39
.............................
20555
Q3MI39
.............................
20556
Q3MI39
.............................
20557
Q3MI39
.............................
20558
Q3MI39
.............................
20559
Q3MI39
.............................
20560
Q3MI3

20728
Q6IPL9
.............................
20729
Q6IPL9
.............................
20730
Q6IPL9
.............................
20731
Q6IPL9
.............................
20732
Q6IPL9
.............................
20733
Q6IPL9
.............................
20734
Q6IPL9
.............................
20735
Q6IPL9
.............................
20736
Q6IPL9
.............................
20737
Q3B891
.............................
20738
Q3B891
.............................
20739
Q3B891
.............................
20740
Q3B891
.............................
20741
Q3B891
.............................
20742
Q3B891
.............................
20743
Q3B891
.............................
20744
Q3B891
.............................
20745
Q3B891
.............................
20746
Q3B891
.............................
20747
Q3B891
.............................
20748
Q3B891
.............................
20749
Q3B891
.............................
20750
Q3B891
.............................
20751
Q3B89

In [ ]:

#blosum62.update(((b,a),val) for (a,b),val in blosum62.items())
#blosum62


In [ ]:
#aligner = Align.PairwiseAligner()
#aligner.substitution_matrix = blosum62
#aligner.mode = 'local'
#aligner.open_gap_score = -10
#aligner.extend_gap_score = -0.5
#alignments = alignelr.align(query_seq[0].seq, target_seq[0].seq)

#print(alignments[0].score)
#print(aligner)

In [6]:
df=pd.read_csv('../Data/NpInter2 Interacting Protiens/Protein-Protein Sequence Similarity/Protien-Protien_Similarity.csv')
df.shape

(20880, 6)

In [7]:
df.nunique()

Query_Seq_ID        145
Target_Seq_ID       145
Align_Score         399
Normalize_Score      45
Identity           1397
Similarity         1843
dtype: int64

In [8]:
df_pro=df[df['Align_Score']>=30]

df_pro.shape

(19384, 6)

In [9]:
df_pro.nunique()

Query_Seq_ID        145
Target_Seq_ID       145
Align_Score         374
Normalize_Score      45
Identity           1390
Similarity         1840
dtype: int64

In [10]:
df_pro['Align_Score'].max()

3900.5

In [11]:
df_pro.head()

,Query_Seq_ID,Target_Seq_ID,Align_Score,Normalize_Score,Identity,Similarity
0,Q9UPQ9,Q13148,118.5,0.02,24.11%,34.11%
1,Q9UPQ9,P08579,73.0,0.02,24.76%,36.19%
2,Q9UPQ9,O14746,81.5,0.01,21.54%,31.49%
3,Q9UPQ9,Q9UHB9,59.0,0.01,30.00%,51.66%
4,Q9UPQ9,O75940,67.0,0.01,21.66%,31.25%
